In [1]:
-- https://karoyakani.wordpress.com/2014/06/27/inside-the-convex-hull/
import Data.List
import Test.QuickCheck
 
type Point      = [Integer]
type Sign       = Int
type Simplex    = ([Point], Sign)
type Facet      = Simplex
 
dimension :: Point -> Int
dimension = pred . length
 
orientation :: [Point] -> Sign
orientation = fromIntegral . signum . det
 
-- determinants
det :: [[Integer]] -> Integer
det [[x]]   = x
det xs      = foldr1 (-) (zipWith (*) col1 (map det (minors cols)))
    where
    col1 = map head xs
    cols = map tail xs
 
minors :: [a] -> [[a]]
minors []       = []
minors (x:xs)   = xs : map (x:) (minors xs)
 
-- a simplex of a lower dimension
facets :: Simplex -> [Facet]
facets (ps, b) = zip (minors ps) (cycle [b, -b])
 
-- is a point inside of a convex simplex?
insideCS :: Simplex -> Point -> Bool
insideCS s p = and [b*orientation (p:us) >= 0 | (us, b) <- facets s]
 
-- convex hull
insideCH :: [Point] -> Point -> Bool
insideCH vs p = or [insideCS s p | s <- simplexes vs]
 
simplexes :: [Point] -> [Simplex]
simplexes vs = [(us, b) | us <- tuples (d+1) vs, let b = orientation us, b /= 0]
    where d = dimension (head vs)
 
tuples :: Int -> [a] -> [[a]]
tuples n = filter ((==n) . length) . subsequences
 
-- an incremental algorithm
insideCH' :: [Point] -> Point -> Bool
insideCH' vs p = or [insideCS s p | s <- partition' vs]
 
partition' :: [Point] -> [Simplex]
partition' vs = case findSimplex vs of
    Nothing -> []
    Just s  -> foldl update [s] (vs \\ vertices s)
 
vertices :: Simplex -> [Point]
vertices = sort . fst
 
-- find a simplex
findSimplex :: [Point] -> Maybe Simplex
findSimplex vs = if null s then Nothing else Just (head s)
    where s = simplexes vs
 
-- update for folding
external :: [Simplex] -> [Facet]
external = foldr op [] . sort . concatMap facets
 
op :: Facet -> [Facet] -> [Facet]
op s []     = [s]  -- bug fixed
op s (t:ss) = if vertices s == vertices t then ss else s:t:ss
 
visible :: Point -> [Facet] -> [Facet]
visible v fs = [(us, b) | (us,b) <- fs, b*orientation (v:us) < 0]
 
newSimplex :: Point -> Facet -> Simplex
newSimplex v (us, b) = (v:us, -b)
 
update :: [Simplex] -> Point -> [Simplex]
update s v = s ++ map (newSimplex v) (visible v (external s))
 
-- an improvement
insideCH'' vs p = if null fs
    then False
    else and [0 <= b*orientation (p:us) | (us, b) <- fs]
    where fs = faces vs
 
faces :: [Point] -> [Facet]
faces vs = case findSimplex vs of
    Nothing -> []
    Just s  -> foldl update' (facets s) (vs \\ vertices s)
 
update' :: [Facet] -> Point -> [Facet]
update' fs v = (fs \\ fs') ++ map (newFacet v) (external fs')
    where fs' = visible v fs
 
newFacet v (us, b) = (v:us, b)
 
-- QuickCheck
point :: Int -> Gen [Integer]
point d = do { xs <- vector d; return (xs ++ [1]) }
 
points :: Int -> Int -> Gen [[Integer]]
points d 0  = return []
points d n  = do { p <- point d; ps <- points d (n-1); return (p:ps) }
 
prop_Hull :: Int -> Int -> Property
prop_Hull d n =
    forAll (points d n) $ \vs ->
    forAll (point d) $ \v ->
    insideCH vs v == insideCH' vs v && insideCH vs v == insideCH'' vs v
 
qc = quickCheck (prop_Hull 2 4)

Line 82: Redundant if
Found:
if null fs then False else
  and [0 <= b * orientation (p : us) | (us, b) <- fs]
Why not:
not (null fs) &&
  and [0 <= b * orientation (p : us) | (us, b) <- fs]

In [2]:
qc

+++ OK, passed 100 tests.

In [3]:
pts = [[2, 4, 1],
       [4, 7, 1],
       [2, 2, 1],
       [9, 6, 1],
       [1, 9, 1],
       [5, 7, 1],
       [0, 9, 1],
       [5, 1, 1],
       [0, 2, 1],
       [8, 4, 1],
       [6, 6, 1],
       [3, 6, 1],
       [1, 9, 1],
       [0, 5, 1],
       [0, 3, 1],
       [9, 4, 1],
       [9, 5, 1],
       [6, 0, 1],
       [6, 7, 1],
       [3, 6, 1],
       [6, 8, 1],
       [1, 0, 1],
       [7, 8, 1],
       [9, 6, 1],
       [6, 9, 1],
       [2, 9, 1],
       [8, 8, 1],
       [6, 7, 1],
       [9, 8, 1],
       [0, 4, 1]]

In [ ]:
-- [0, 1, 1] [50, 50, 1]
-- testPts = [[5, 5, 1], [0, 1, 1], [1, 4, 1], [50, 50, 1]]
testPts = [[-5, -4, 1]]

r = map(insideCH pts) testPts
r

In [5]:
testPts = [[-5, -4, 1]]
r = map(insideCH' pts) testPts
r

[False]